# Mosaic AI Agent Framework: Author and deploy a tool-calling DSPy agent
This notebook demonstrates how to author a DSPy agent that's compatible with Mosaic AI Agent Framework features. In this notebook you learn to:

Author a tool-calling DSPy agent wrapped with ChatAgent
Manually test the agent's output

Evaluate the agent using Mosaic AI Agent Evaluation
Log and deploy the agent
To learn more about authoring an agent using Mosaic AI Agent Framework, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/author-agent) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/create-chat-model)).

Prerequisites
Address all TODOs in this notebook.

In [0]:
%pip install dspy mlflow unitycatalog-ai[databricks] pyjokes
dbutils.library.restartPython()


## Define the agent in code
Define the agent code in a single cell below. This lets you easily write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.

#### Agent tools
This agent code adds the built-in Unity Catalog function `system.ai.python_exec` to the agent. The agent code also includes commented-out sample code for adding a vector search index to perform unstructured data retrieval.

For more examples of tools to add to your agent, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool))

#### Wrap the DSPy agent using the `ChatAgent` interface

DSPy is a lightweight, pure python framework. It does not use framework specific objects to be compatible with other libraries. 

As such, you can simply use Databricks functionality as is via the SDK, Unity Catalog AI or MLflow

Databricks recommends using `ChatAgent` as it simplifies authoring multi-turn conversational agents using an open source standard. See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).



In [0]:
import mlflow
mlflow_experiment_path = "/Users/austin.choi@databricks.com/dspy_pyfunc_deploy"
mlflow.set_experiment(experiment_name=mlflow_experiment_path)

# Get the experiment ID to use in the next step
experiment_id = mlflow.tracking.fluent._get_experiment_id()
print(experiment_id)


In [0]:
import dspy
import mlflow
llama = dspy.LM('databricks/databricks-meta-llama-3-3-70b-instruct', cache=False)
# llama = dspy.LM('databricks/databricks-meta-llama-3-1-8b-instruct', cache=False)
# claude = dspy.LM('databricks/databricks-claude-3-7-sonnet', cache=False)
dspy.configure(lm=llama)
mlflow.dspy.autolog()

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

# Initialize the client, connecting to serverless
client = DatabricksFunctionClient(execution_mode="local")

# Define the function
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers and returns the result.

    Args:
        a (float): First number.
        b (float): Second number.

    Returns:
        float: The sum of the two numbers.
    """
    return a + b

# Create the function in UC
# client.create_python_function(
#     func=add_numbers,
#     catalog="austin_choi_demo_catalog",
#     schema="agents"
# )
tools =["austin_choi_demo_catalog.agents.add_numbers","system.ai.python_exec"]

# Execute the function
# result = client.execute_function(
#     "austin_choi_demo_catalog.agents.add_numbers",
#     parameters={"a": 10.5, "b": 5.5}
# )

result = client.execute_function(
    "system.ai.python_exec",
    parameters={ "code": "import pyjokes; print(pyjokes.get_joke())"}
)

print(result.value)  # Outputs: 16.0

In [0]:
my_func_def = client.get_function_source(function_name=f"system.ai.python_exec")
print(my_func_def)

In [0]:
functions = client.list_functions(catalog="austin_choi_demo_catalog", schema="agents", max_results=10)
functions

In [0]:
def uc_function_executor(uc_tool, parameter): 
  result = client.execute_function(
    uc_tool,
    parameters=parameter
)
  return result.value


In [0]:
def get_uc_information(uc_tool): 
  my_func_def = client.get_function_source(function_name=uc_tool)
  return my_func_def


In [0]:
import dspy
from typing import Literal
class uc_function_selector(dspy.Signature):
  """
  Follow this routine:
  1. Given list of unity_catalog_tools, pick the right unity_catalog_tool for the question 
  2. Use the get_uc_information to understand what the unity_catalog_tool does. Find what parameters the function expects
  3. Change the question to a compatible input according to the parameter to make it compatible with the unity_catalog_tool based on get_uc_information. Double check typing of the input. The input looks like this {parameter_name: value compatible with parameter}
  4. Then execute the tool using uc_function_executor
  """ 
  question: str = dspy.InputField()
  unity_catalog_tools: list = dspy.InputField()
  response: str = dspy.OutputField() 

In [0]:
execute_unity_catalog = dspy.ReAct(uc_function_selector, tools=[get_uc_information,uc_function_executor],max_iters=2)

In [0]:
# question = "add these numbers 2.0 and 3.0"
# question = "Use Python to figure out how many rs are in the word strrawberry"
question = "Use Python, import emoji, and emojize pything is :thumbs_up"
tools =["austin_choi_demo_catalog.agents.add_numbers","system.ai.python_exec"]
result = execute_unity_catalog(question=question, unity_catalog_tools=tools)
result.response

In [0]:
%%writefile agent.py

from typing import Any, Generator, Optional
from databricks.sdk.service.dashboards import GenieAPI
import mlflow
from databricks.sdk import WorkspaceClient
from mlflow.entities import SpanType
from mlflow.pyfunc.model import ChatAgent
from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
import dspy
import uuid

# Autolog DSPy traces to MLflow
mlflow.dspy.autolog()

# Set up DSPy with a Databricks-hosted LLM
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
lm = dspy.LM(model=f"databricks/{LLM_ENDPOINT_NAME}")
dspy.settings.configure(lm=lm)

class genie_selector_agent(dspy.Signature):
  """
  Given the sql_instructions, determine which genie space tool to call, send the exact sql_instruction text to the tool and answer the question given the response from the tool.
  """ 
  sql_instruction: str = dspy.InputField()
  response: str = dspy.OutputField() 
  sql_query_output:  list = dspy.OutputField()

class DSPyChatAgent(ChatAgent):     
    def __init__(self):
      self.genie_selector_agent = genie_selector_agent
      self.multi_genie_agent = dspy.ReAct(self.genie_selector_agent, tools=[self.hls_patient_genie, self.investment_portfolio_genie], max_iters=1)

    def hls_patient_genie(self, sql_instruction):

      w = WorkspaceClient()
      genie_space_id = "01effef4c7e113f9b8952cf568b49ac7"

      # Start a conversation
      conversation = w.genie.start_conversation_and_wait(
          space_id=genie_space_id,
          content=f"{sql_instruction} always limit to one result"
      )

      response = w.genie.get_message_attachment_query_result(
        space_id=genie_space_id,
        conversation_id=conversation.conversation_id,
        message_id=conversation.message_id,
        attachment_id=conversation.attachments[0].attachment_id
      )

      return response.statement_response.result.data_array

    def investment_portfolio_genie(self, sql_instruction):

      w = WorkspaceClient()
      genie_space_id = "01f030d91cc6165d88aaee122a274294"

      # Start a conversation
      conversation = w.genie.start_conversation_and_wait(
          space_id=genie_space_id,
          content=f"{sql_instruction} always limit to one result"
      )

      response = w.genie.get_message_attachment_query_result(
        space_id=genie_space_id,
        conversation_id=conversation.conversation_id,
        message_id=conversation.message_id,
        attachment_id=conversation.attachments[0].attachment_id
      )

      return response.statement_response.result.data_array


    def prepare_message_history(self, messages: list[ChatAgentMessage]):
        history_entries = []
        # Assume the last message in the input is the most recent user question.
        for i in range(0, len(messages) - 1, 2):
            history_entries.append({"question": messages[i].content, "answer": messages[i + 1].content})
        return dspy.History(messages=history_entries)

    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        latest_question = messages[-1].content
        response = self.multi_genie_agent(sql_instruction=latest_question).response
        return ChatAgentResponse(
            messages=[ChatAgentMessage(role="assistant", content=response, id=uuid.uuid4().hex)]
        )

# Set model for logging or interactive testing
from mlflow.models import set_model
AGENT = DSPyChatAgent()
set_model(AGENT)

In [0]:
import mlflow
from agent import LLM_ENDPOINT_NAME
from mlflow.models.resources import (
    DatabricksFunction,
    DatabricksGenieSpace,
    DatabricksServingEndpoint,
)
from pkg_resources import get_distribution

resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksGenieSpace(genie_space_id="01f030d91cc6165d88aaee122a274294"),
    DatabricksGenieSpace(genie_space_id="01effef4c7e113f9b8952cf568b49ac7"),
]

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        # input_example=input_example,
        extra_pip_requirements=[f"databricks-connect=={get_distribution('databricks-connect').version}"],
        resources=resources,
    )